In [1]:
import pandas as pd
import ast
import re
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import numpy as np
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

# get Indonesian stopword 
list_stopwords = set(stopwords.words('indonesian'))

# factory = StopWordRemoverFactory()
# stopword = factory.create_stop_word_remover()
# fact = StemmerFactory()
# stemmer = fact.create_stemmer()

In [2]:
df = pd.read_csv('./dataset/database_konten_fixed.csv')
df.head()

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes
0,61a420e4-0044-11ee-bbbb-bb155eb672e5,Tips Menjaga Kebugaran Lansia yang Perlu Diket...,"“Seiring dengan bertambahnya umur, menjaga keb...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...",dr. Rizal Fadli,29 Mei 2023,"[""kesehatan"",""lansia"",""kebugaran""]",https://www.halodoc.com/artikel/tips-menjaga-k...,0,0
1,61b5264f-0044-11ee-ac36-bb155eb672e5,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri ...",“Ada berbagai manfaat seledri untuk kesehatan....,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seledri merupakan sayuran y...",dr. Fadhli Rizal Makarim,30 Mei 2023,"[""kesehatan"",""makanan""]",https://www.halodoc.com/artikel/jarang-diketah...,0,0
2,61c727ae-0044-11ee-bd96-bb155eb672e5,9 Obat Herbal yang Ampuh Atasi Sakit Lutut dan...,“Obat herbal dapat menjadi pilihan alternatif ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Sakit lutut dan sendi adala...",dr. Rizal Fadli,10 Mei 2023,"[""herbal"",""kesehatan"",""obat""]",https://www.halodoc.com/artikel/9-obat-herbal-...,0,0
3,61dae8d1-0044-11ee-ac0f-bb155eb672e5,5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula,NaN,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,“Yoga adalah olahraga yang terbilang cukup rin...,dr. Rizal Fadli,03 Desember 2021,"[""olahraga"",""yoga"",""pemula""]",https://www.halodoc.com/artikel/5-gerakan-yoga...,0,0
4,61eb98ae-0044-11ee-a45a-bb155eb672e5,Lakukan 5 Gerakan Yoga Ini untuk Meredakan Stres,NaN,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Yoga dapat membantu mengura...",dr. Verury Verona Handayani,18 Desember 2020,"[""yoga"",""olahraga"",""kesehatan mental"",""stres""]",https://www.halodoc.com/artikel/lakukan-5-gera...,0,0


In [3]:
df['tags'] = df['tags'].apply(lambda x: " ".join(ast.literal_eval(x)))

same_summary = []
for i in range(df.shape[0]):
    if df.loc[i,'summary'] not in same_summary:
        same_summary.append(df.loc[i,'summary'])
    else:
        df.loc[i,'summary'] = ' '

df.fillna(' ',inplace=True)

df['text'] = df['title']+' '+df['summary']+' '+df['content'] + ' ' + df['tags']

def clean_text(x):
    x = x.lower()
    x = x.strip()
    x = re.sub('[0-9!?@()\\xa0\\r\\n.“”,:|]',' ',x)
    x = [word.strip() for word in x.split(' ') if not word in list_stopwords if word != '']
    return x

In [4]:
corpus= df['text'].apply(clean_text)

In [5]:
# Step 4: Compute user-feed similarity
def calculate_user_feed_similarity(user_embedding, feed_embeddings):
    return cosine_similarity(user_embedding.reshape(1, -1), list(feed_embeddings.values()))[0]

In [6]:
model_pretrained = Word2Vec.load('./word2vec-pretrained-model/wiki.id.case.model')
feed_embeddings = {feed_id: sum(model_pretrained.wv[word] for word in text.split() if word in model_pretrained.wv)
                   for feed_id, text in df[['id', 'text']].values}

In [1]:
# Step 5: Generate recommendations for a user based on user input text
user_input_text = 'saya mengalami stress dan merasakan kesehatan mental terganggu'
user_embedding = sum(model_pretrained.wv[word] for word in user_input_text.split() if word in model_pretrained.wv)
user_feed_similarity = calculate_user_feed_similarity(user_embedding, feed_embeddings)

NameError: name 'model_pretrained' is not defined

In [24]:
user_feed_similarity = calculate_user_feed_similarity(user_embedding, feed_embeddings)

In [25]:
# Find most relevant feeds to the user based on similarity scores
relevant_feeds_indices = user_feed_similarity.argsort()[-5:][::-1]  # Get top 3 relevant feeds
recommended_feeds = [list(feed_embeddings.keys())[i] for i in relevant_feeds_indices]

df[df['id'].isin(recommended_feeds)]

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes,text
22,63366c54-0044-11ee-a3b3-bb155eb672e5,8 Makanan Sehat untuk Menjaga Kesehatan Keluarga,“Rutin mengonsumsi makanan sehat mampu memenuh...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Makanan sehat penting untuk...",dr. Rizal Fadli,10 April 2023,makanan makanan sehat keluarga kesehatan,https://www.halodoc.com/artikel/8-makanan-seha...,0,0,8 Makanan Sehat untuk Menjaga Kesehatan Keluar...
31,63cee60e-0044-11ee-a041-bb155eb672e5,5 Rekomendasi Makanan untuk Diet Sehat yang Pa...,“Salah satu cara efektif diet sehat adalah men...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Diet adalah perubahan pola ...",dr. Rizal Fadli,10 Oktober 2022,makanan makanan sehat kesehatan diet,https://www.halodoc.com/artikel/5-rekomendasi-...,0,0,5 Rekomendasi Makanan untuk Diet Sehat yang Pa...
32,63de2139-0044-11ee-a403-bb155eb672e5,"Kaya Nutrisi, Ini 4 Pilihan Makanan Sehat untu...",“Makanan sehat untuk lansia harus yang bernutr...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...",dr. Rizal Fadli,07 Oktober 2022,makanan makanan sehat kesehatan lansia,https://www.halodoc.com/artikel/kaya-nutrisi-i...,0,0,"Kaya Nutrisi, Ini 4 Pilihan Makanan Sehat untu..."
36,64215258-0044-11ee-b842-bb155eb672e5,"Catat, Ini 4 Makanan Diet Sehat untuk Menurunk...","“Saat ingin menurunkan berat badan, kamu perlu...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Salah satu cara ampuh untuk...",dr. Fadhli Rizal Makarim,06 Juli 2022,makanan makanan sehat kesehatan diet,https://www.halodoc.com/artikel/catat-ini-4-ma...,0,0,"Catat, Ini 4 Makanan Diet Sehat untuk Menurunk..."
37,644b6d34-0044-11ee-93a3-bb155eb672e5,Ini Menu Makanan Sehat untuk Keluarga yang Bis...,“Menyajikan menu makanan sehat merupakan cara ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Untuk menjaga kesehatan kel...",dr. Rizal Fadli,20 Mei 2022,makanan makanan sehat kesehatan keluarga,https://www.halodoc.com/artikel/ini-menu-makan...,0,0,Ini Menu Makanan Sehat untuk Keluarga yang Bis...


In [3]:
from recommendation import Recommendation

In [4]:
person = Recommendation('./dataset/database_konten_fixed.json','saya mengalami stress dan merasakan kesehatan mental terganggu')
person.recomendations()

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes
180,6f04d332-0044-11ee-8c81-bb155eb672e5,Kenali 4 Manfaat Stress Ball untuk Hilangkan R...,,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"“Saat mengalami kecemasan, tak sedikit orang y...",dr. Fadhli Rizal Makarim,12 Agustus 2021,cemas psikologi,https://www.halodoc.com/artikel/kenali-4-manfa...,0,0
59,65d8f65b-0044-11ee-99e9-bb155eb672e5,"Penting, Ini 6 Cara Jaga Kesehatan Mental di A...",“Ada beberapa cara yang direkomendasikan untuk...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Kini tahun baru 2023 tingga...",dr. Rizal Fadli,30 Desember 2022,kesehatan mental,https://www.halodoc.com/artikel/penting-ini-6-...,0,0
67,6660ca01-0044-11ee-aaa3-bb155eb672e5,Ini Pentingnya Menjaga Kesehatan Mental Remaja,“Kesehatan mental pada remaja perlu menjadi pe...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Banyak orangtua yang tidak ...",dr. Fadhli Rizal Makarim,25 November 2022,kesehatan mental remaja,https://www.halodoc.com/artikel/ini-pentingnya...,0,0
100,68c36d09-0044-11ee-b57a-bb155eb672e5,Ini Alasan Gen Z Lebih Terbuka Soal Kesehatan ...,“Keterbukaan akan informasi melalui teknologi ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Membicarakan masalah keseha...",dr. Fadhli Rizal Makarim,10 Oktober 2022,kesehatan mental remaja milenial,https://www.halodoc.com/artikel/ini-alasan-gen...,0,0
42,649ed850-0044-11ee-9384-bb155eb672e5,Mengenali Definisi Tepat dari Kesehatan Mental...,“Artikel ini mengajak untuk memahami definisi ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Setiap tahun, dunia memperi...",dr. Rizal Fadli,26 Mei 2023,kesehatan kesehatan mental,https://www.halodoc.com/artikel/mengenali-defi...,0,0
...,...,...,...,...,...,...,...,...,...,...,...
183,6f377412-0044-11ee-959d-bb155eb672e5,"Ibu Tak Perlu Cemas, Ini Cara Atasi Bayi Rewel...",“Kebanyakan bayi rewel di malam hari bukan dis...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Bagi ibu yang baru memiliki...",dr. Rizal Fadli,10 November 2022,cemas wanita anak,https://www.halodoc.com/artikel/ibu-tak-perlu-...,0,0
136,6b740701-0044-11ee-878b-bb155eb672e5,Dampak Negatif Marahi Anak di Depan Orang Lain,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Pola asuh orangtua menentuk...",dr. Fadhli Rizal Makarim,10 Oktober 2019,marah psikologi anak,https://www.halodoc.com/artikel/dampak-negatif...,0,0
138,6b9b8f47-0044-11ee-9725-bb155eb672e5,"Anak Mudah Marah, Begini Tips Menghadapinya",,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Tidak hanya orang dewasa, a...",dr. Fadhli Rizal Makarim,14 Januari 2020,marah psikologi anak,https://www.halodoc.com/artikel/anak-mudah-mar...,0,0
142,6be019bb-0044-11ee-89bf-bb155eb672e5,"Si Kecil Sering Marah, Ini Cara Mengatasinya",,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Ketika anak marah, hal ters...",dr. Verury Verona Handayani,10 Desember 2019,marah psikologi anak,https://www.halodoc.com/artikel/si-kecil-serin...,0,0


In [169]:
# # Step 4: Compute user-feed similarity
# def calculate_user_feed_similarity(user_embedding, feed_embeddings):
#     return cosine_similarity(user_embedding.reshape(1, -1), list(feed_embeddings.values()))[0]
# model_pretrained = Word2Vec.load('./word2vec-pretrained-model/wiki.id.case.model')
# feed_embeddings = {feed_id: sum(model_pretrained.wv[word] for word in text.split() if word in model_pretrained.wv)
#                    for feed_id, text in df[['id', 'text']].values}

# # Step 5: Generate recommendations for a user based on user input text
# user_input_text = 'babi'
# user_embedding = sum(model_pretrained.wv[word] for word in user_input_text.split() if word in model_pretrained.wv)
# user_feed_similarity = calculate_user_feed_similarity(user_embedding, feed_embeddings)

# # Find most relevant feeds to the user based on similarity scores
# relevant_feeds_indices = user_feed_similarity.argsort()[-3:][::-1]  # Get top 3 relevant feeds
# recommended_feeds = [list(feed_embeddings.keys())[i] for i in relevant_feeds_indices]

# df[df['id'].isin(recommended_feeds)]

In [189]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer()
# tfidf_matrix = tfidf.fit_transform(' '.join(corpus.apply(lambda x: ' '.join(x)).tolist()).split(' '))
# # Load pretrained Word2Vec model
# word2vec_model = Word2Vec.load('./word2vec-pretrained-model/wiki.id.case.model')

# # Function to get document vector using TF-IDF
# def get_tfidf_vector(doc):
#     doc_vector = tfidf.transform([doc])
#     return doc_vector

# # Function to get document vector using Word2Vec
# def get_word2vec_vector(doc):
#     doc_vector = np.zeros(400)
#     count = 0
#     for word in doc:
#         if word in word2vec_model.wv:
#             doc_vector += word2vec_model.wv[word]
#             count += 1
#     if count > 0:
#         doc_vector /= count
#     return doc_vector

# # Step 4: Compute user-feed similarity
# def calculate_user_feed_similarity(user_embedding, feed_embeddings, tfidf_vector):
#     tfidf_weights = tfidf_vector.toarray().flatten()
#     tfidf_weights /= np.linalg.norm(tfidf_weights)
#     feed_embeddings_array = np.array(list(feed_embeddings.values()))
#     feed_embeddings_array = np.multiply(feed_embeddings_array, tfidf_weights[:, np.newaxis])
#     user_embedding = np.multiply(user_embedding, tfidf_weights.reshape(-1, 1))
#     return cosine_similarity(user_embedding, feed_embeddings_array).flatten()


# # Generate feed embeddings using Word2Vec model
# feed_embeddings = {'id': get_word2vec_vector(x) for x in corpus}

# # Function to recommend feeds for a user based on user input text
# def recommend_feeds(user_input_text, num_feeds):
#     user_input_doc = clean_text(user_input_text)
#     user_tfidf_vector = get_tfidf_vector(user_input_text)
#     user_word2vec_vector = get_word2vec_vector(user_input_doc)

#     user_feed_similarity = calculate_user_feed_similarity(user_word2vec_vector, feed_embeddings, user_tfidf_vector)
#     relevant_feed_indices = np.argsort(user_feed_similarity)[::-1][:num_feeds]
#     recommended_feeds = [list(feed_embeddings.keys())[i] for i in relevant_feed_indices]
#     return recommended_feeds

In [216]:
# feed_embeddings_array

NameError: name 'feed_embeddings_array' is not defined

In [193]:
# user_input_doc = clean_text(user_input_text)
# user_tfidf_vector = get_tfidf_vector(user_input_text)
# user_word2vec_vector = get_word2vec_vector(user_input_doc)

# user_feed_similarity = calculate_user_feed_similarity(user_word2vec_vector, feed_embeddings, user_tfidf_vector)
# relevant_feed_indices = np.argsort(user_feed_similarity)[::-1]
# # recommended_feeds = [list(feed_embeddings.keys())[i] for i in relevant_feed_indices]

In [198]:
# relevant_feed_indices

array([31507786, 23790795, 31508043, ..., 35604573, 35604572,        0],
      dtype=int64)

In [117]:
relevant_feed_indices

array([399, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137,
       138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 126, 124, 149,
       123, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 148, 150, 100, 176,
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
       191, 192, 193, 194, 195, 196, 197, 198, 177, 175, 151, 174, 152,
       153, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 169, 170, 171, 172, 173, 101,  99, 200,  25,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  26,  23,  49,  22,   1,   2,   3,
         4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,
        17,  18,  19,  20,  21,  48,  50,  98,  75,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  76,  74,  51,  73,  52,  53,  54,  5

In [116]:
recommended_feeds = [list(feed_embeddings.keys())[i] for i in relevant_feed_indices]

IndexError: list index out of range

In [111]:
relevant_feed_indices

NameError: name 'relevant_feed_indices' is not defined

In [109]:
df[df['id'].isin(recommended_feeds)]

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes,text
42,649ed850-0044-11ee-9384-bb155eb672e5,Mengenali Definisi Tepat dari Kesehatan Mental...,“Artikel ini mengajak untuk memahami definisi ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Setiap tahun, dunia memperi...",dr. Rizal Fadli,26 Mei 2023,kesehatan kesehatan mental,https://www.halodoc.com/artikel/mengenali-defi...,0,0,Mengenali Definisi Tepat dari Kesehatan Mental...
58,65acd29f-0044-11ee-b351-bb155eb672e5,Ini Manfaat Memiliki Sikap Realistis untuk Kes...,"""Sikap realistis adalah cara berpikir yang pen...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Banyak yang berpikir bahwa ...",dr. Fadhli Rizal Makarim,20 Januari 2023,kesehatan mental psikologi,https://www.halodoc.com/artikel/ini-manfaat-me...,0,0,Ini Manfaat Memiliki Sikap Realistis untuk Kes...
76,66f7d720-0044-11ee-b9c8-bb155eb672e5,3 Cara Efektif Menjaga Kesehatan Mental pada R...,“Menjaga kesehatan mental remaja sangat pentin...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Banyak faktor yang mempenga...",dr. Fadhli Rizal Makarim,04 November 2022,kesehatan mental remaja,https://www.halodoc.com/artikel/3-cara-efektif...,0,0,3 Cara Efektif Menjaga Kesehatan Mental pada R...


In [106]:
# Example usage: Recommend 5 feeds for user based on user input text
user_input_text = "saya ingin menjaga kesehatan mental saya lebih baik lagi"
recommended_feeds = recommend_feeds(user_input_text, 1)
print(recommended_feeds)

IndexError: list index out of range